In [1]:
import os 
BASE_DIR=os.path.abspath("../../")

In [2]:
import sys 
sys.path.append(BASE_DIR)
DATA_DIR="/home/arturosf/Documentos/repos/DeepElasticity/data/001-LinearElasticity"

In [3]:
from src.utils.data import Data
from src.PINN import PINN
from src.PINN_lame import PINN_lame 

from src.train import Trainer
import torch 
import matplotlib.pyplot as plt 

In [6]:
data=Data("MULTIPLE_E_VALUES",10,E="0.032",base_dir=DATA_DIR)
data.load_data()

In [7]:
train_init_pos_main,train_disp_main,test_init_pos_main,test_disp_main,position_selected_stresses,return_stress,return_colloc_points=data.prepare_pytorch_data()

/home/arturosf/miniconda3/envs/pinns/lib/python3.10/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [28]:
init_values={"nu":.4,"alpha":1.1,"E_ref":0.05}
pinn_real=PINN([3,80,80,80,80,80,80,3],init_values,loss_weights_init={"data":1,"PDE":1,"BC":1})
step_dict = {
    "step_1": {"optim": torch.optim.Adam(pinn_real.parameters(), lr=1e-3), 
               "epochs": 17}
    }

/home/arturosf/Documentos/repos/DeepElasticity/src/PINN.py:93: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.E = torch.tensor((1+self.alpha)*self.E_ref,dtype=torch.float32).to(self.device)


In [27]:
pinn=PINN_lame([3,20,20,20,3], init_values={"lambda": 0.01, "mu": 2}, 
          train_lambda=True,train_mu=True, device=None, separate_data_losses=True, 
          loss_weights_init={"data": 1, "PDE": 1, "BC": 1})

TypeError: PINN.__init__() got an unexpected keyword argument 'train_lambda'

In [12]:

step_dict = {
    "step_1": {"optim": torch.optim.Adam(pinn.parameters(), lr=1e-3), 
               "epochs": 17}
    }
trainer=Trainer(steps_dict=step_dict)

In [16]:
pinn(train_init_pos_main)

tensor([[-0.5114, -0.3636,  0.4230],
        [-0.4948, -0.4088,  0.4539],
        [-0.4948, -0.4088,  0.4540],
        ...,
        [ 0.1796,  0.8211,  0.1343],
        [-0.4898, -0.4011,  0.4547],
        [-0.5449,  0.4933,  0.0859]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [29]:

loss=pinn.loss_function(train_disp_main,pinn_real(train_init_pos_main))
loss.backward()

In [31]:
list(pinn_real.parameters())[0].grad

In [32]:
list(pinn_real.parameters())

[Parameter containing:
 tensor(1.1000, device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[ 0.1153, -0.0148,  0.1366],
         [-0.0965,  0.1301, -0.1193],
         [-0.2546, -0.0047,  0.1506],
         [ 0.1760,  0.0788, -0.2529],
         [ 0.2193,  0.1704,  0.1013],
         [-0.1503,  0.0208, -0.1039],
         [-0.2048, -0.1465,  0.0999],
         [ 0.0041, -0.0298, -0.0593],
         [-0.0965, -0.0641,  0.0382],
         [-0.2266, -0.0615, -0.2639],
         [ 0.1735, -0.2749, -0.3167],
         [ 0.0816, -0.0472,  0.1542],
         [ 0.2478, -0.0861, -0.1067],
         [ 0.0191,  0.0989, -0.0319],
         [ 0.0671, -0.0762, -0.0037],
         [-0.1111, -0.2142,  0.0678],
         [ 0.2646,  0.0290,  0.1700],
         [-0.0911,  0.0364, -0.0024],
         [-0.0305,  0.0764, -0.2015],
         [-0.2745,  0.1408, -0.0976],
         [-0.6004, -0.1561, -0.0690],
         [ 0.0803, -0.0506, -0.0145],
         [ 0.0451, -0.1095,  0.0868],
         [-0.1626, -0.2306

In [ ]:
trainer.train(pinn,data)